In [2]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration
import sentencepiece
import random
import torch

In [3]:
# Global constants
SEED = 42 # Random seed for reproducability
random.seed(SEED)

In [4]:
dataset = load_dataset("allenai/qasper")

Found cached dataset qasper (/home/nano/.cache/huggingface/datasets/allenai___qasper/qasper/0.3.0/2bfcd239e581ab83f9ab7b76a82e42c6bcf574a13246ae6cc5a6c357c35f96f9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
def get_formatted_dataset_from_split(dataset_split):
    abstracts = []
    questions = []
    answers = []
    
    # Iterate over all articles
    for article in dataset_split:
        qa = article['qas']

        # Iterate over all questions and answers
        for question, answer in zip(qa['question'], qa['answers']):
            # Generate all answer candidates, from which we then randomly sample
            answer_candidates = []
            for question_answer in answer['answer']:              
                answer = question_answer['free_form_answer'] if question_answer['free_form_answer'] else ' '.join(question_answer['extractive_spans'])
                answer_candidates.append(answer)

            # Finally add relevant objects to training data
            abstracts.append(article['abstract'])
            questions.append(question)
            answers.append(random.choice(answer_candidates)) # Randomly sample from available answers
    
    # DEBUG: Sanity check 
    assert len(abstracts) == len(questions) == len(answers)
    
    return Dataset.from_dict(
        {'abstract': abstracts,
        'question': questions,
        'answer': answers}
    )

In [6]:
# Task 4 - Initial Preprocessing of qasper dataset
train_dataset = get_formatted_dataset_from_split(dataset['train'])
test_dataset = get_formatted_dataset_from_split(dataset['test'])

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['abstract', 'question', 'answer'],
    num_rows: 2593
})
Dataset({
    features: ['abstract', 'question', 'answer'],
    num_rows: 1451
})


In [7]:
# Task 5 - Get validation set from train
train_val_dataset = train_dataset.train_test_split(test_size=0.1, seed=SEED)

dataset = DatasetDict({
    'train': train_val_dataset["train"],
    'test': test_dataset,
    'val': train_val_dataset['test']
})

dataset

DatasetDict({
    train: Dataset({
        features: ['abstract', 'question', 'answer'],
        num_rows: 2333
    })
    test: Dataset({
        features: ['abstract', 'question', 'answer'],
        num_rows: 1451
    })
    val: Dataset({
        features: ['abstract', 'question', 'answer'],
        num_rows: 260
    })
})

In [8]:
# Task 6 - Preprocessing Function
# TODO: Concat question and answer and truncate to 128 tokens
# TODO: Truncate answers to 32 tokens
tokenizer = T5Tokenizer.from_pretrained("t5-small")


In [9]:
# Task 7 - Apply preprocessing function using map

In [10]:
# Task 8 - load google/t5-efficient-tiny model with pre-trained weights

model = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-tiny")

ImportError: 
T5ForConditionalGeneration requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
